In [20]:
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm.notebook import tqdm
import pandas as pd
import time
from tqdm.notebook import tqdm
import pickle

In [2]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Chrome/87.0 (Ismail.Harrando@eurecom.fr) SPARQLWrapper")

In [3]:
classes = {
    "Organization": "Q43229", #includes companies
    "Name": "Q82799",
    "Artist": "Q483501",
    "Geolocation": "Q2221906",
    "City": "Q515",
    "Capital": "Q5119",
    "Town": "Q3957",
    "Demonym": "Q217438",
    "Product": "Q2424752",
    "Brand": "Q431289",
    "Georegion": "Q82794",
    "Country": "Q6256",
}

In [4]:
len(classes)

12

In [23]:
%%time
# From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
results = {}
limit = 20000

for label, wikidata_code in tqdm(classes.items()):
    print(label, wikidata_code)
    results[label] = []
    offset = 0
    while True:
        try:
             #  ?entity ?entityLabel ?entityAltLabel ?entityDescription
            query = """
                    SELECT DISTINCT ?entity ?entityLabel ?entityAltLabel ?entityDescription
                    WHERE
                    {{
                      {{
                        SELECT DISTINCT ?entity WHERE {{
                          ?entity wdt:P31|wdt:P31/wdt:P279|wdt:P31/wdt:P279/wdt:P279 wd:{};
                                  wdt:P31 ?class
                        }}
                        OFFSET {}
                        LIMIT {}
                      }}
                      SERVICE wikibase:label {{ bd:serviceParam wikibase:language 'en' }}
                    }}
                    """.format(wikidata_code, offset*limit, limit)
            if offset == 0:
                print(query)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            offset += 1
            if len(sparql.query().convert()['results']['bindings']) > 0:
                result = sparql.query().convert()['results']['bindings']
                results[label].append(result)
                print(f'{len(results[label][-1])} of results returned at offset {offset}')
            else:
                print(f'No more results returned (offset {offset})')
                break
        except Exception as e:
            print(f'Exception at label {label} (offset {offset}) : {e}')

Organization Q43229

                    SELECT DISTINCT ?entity ?entityLabel ?entityAltLabel ?entityDescription
                    WHERE
                    {
                      {
                        SELECT DISTINCT ?entity WHERE {
                          ?entity wdt:P31|wdt:P31/wdt:P279|wdt:P31/wdt:P279/wdt:P279 wd:Q43229.
                        }
                        OFFSET 0
                        LIMIT 20000
                      }
                      SERVICE wikibase:label { bd:serviceParam wikibase:language 'en' }
                    }
                    
20000 of results returned at offset 1
20000 of results returned at offset 2
20000 of results returned at offset 3
20000 of results returned at offset 4
20000 of results returned at offset 5
20000 of results returned at offset 6
20000 of results returned at offset 7
20000 of results returned at offset 8
20000 of results returned at offset 9
20000 of results returned at offset 10
20000 of results returned at off

20000 of results returned at offset 99
20000 of results returned at offset 100
20000 of results returned at offset 101
20000 of results returned at offset 102
20000 of results returned at offset 103
20000 of results returned at offset 104
20000 of results returned at offset 105
20000 of results returned at offset 106
20000 of results returned at offset 107
20000 of results returned at offset 108
20000 of results returned at offset 109
20000 of results returned at offset 110
20000 of results returned at offset 111
20000 of results returned at offset 112
20000 of results returned at offset 113
20000 of results returned at offset 114
20000 of results returned at offset 115
20000 of results returned at offset 116
20000 of results returned at offset 117
20000 of results returned at offset 118
20000 of results returned at offset 119
20000 of results returned at offset 120
20000 of results returned at offset 121
20000 of results returned at offset 122
20000 of results returned at offset 123
2

20000 of results returned at offset 31
20000 of results returned at offset 32
20000 of results returned at offset 33
20000 of results returned at offset 34
20000 of results returned at offset 35
20000 of results returned at offset 36
20000 of results returned at offset 37
20000 of results returned at offset 38
20000 of results returned at offset 39
20000 of results returned at offset 40
20000 of results returned at offset 41
20000 of results returned at offset 42
20000 of results returned at offset 43
20000 of results returned at offset 44
20000 of results returned at offset 45
20000 of results returned at offset 46
20000 of results returned at offset 47
20000 of results returned at offset 48
20000 of results returned at offset 49
20000 of results returned at offset 50
20000 of results returned at offset 51
20000 of results returned at offset 52
20000 of results returned at offset 53
20000 of results returned at offset 54
20000 of results returned at offset 55
20000 of results returned

In [ ]:
for label in classes:
    print(label.upper())
    print(len(results[label]), 'results')
    print(len(results_2[label]), 'results')
    print(0 if label not in results_3 else len(results_3[label]), 'results')
    print('')

In [25]:
for label in classes:
    print(label.upper())
    print('About', len(results[label]) * limit, 'results')
    print('')

ORGANIZATION
About 940000 results

NAME
About 40000 results

ARTIST
About 20000 results

GEOLOCATION
About 3100000 results

CITY
About 40000 results

CAPITAL
About 20000 results

TOWN
About 60000 results

DEMONYM
About 20000 results

PRODUCT
About 80000 results

BRAND
About 20000 results

GEOREGION
About 1700000 results

COUNTRY
About 20000 results



In [26]:
# v1.0 working with {0,2}
# v1.1 adding classes QID
pickle.dump(results, open('gazetteers_v1.1_results.pickle', 'wb'))

In [ ]:
'''
SELECT distinct  ?entity (GROUP_CONCAT(?class ; SEPARATOR = ';') as ?classes)
WHERE 
{ 
     ?entity wdt:P31/wdt:P279* wd:Q82799;
             wdt:P31 ?class.
}
GROUP BY ?entity
LIMIT 100
'''

In [42]:
%%time
# From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
# results_classes = {}
limit = 10000

for label, wikidata_code in tqdm(classes.items()):
    print(label, wikidata_code)
    if label in ['Organization', 'Name', 'Artist', 'City', "Capital", 'Town', 'Demonym', "Product", 
                 'Brand']:
        continue
    results_classes[label] = []
    offset = 0
    while True:
        try:
             #  ?entity ?entityLabel ?entityAltLabel ?entityDescription
            query = """
                    SELECT distinct  ?entity (GROUP_CONCAT(?class ; SEPARATOR = ';') as ?classes)
                    WHERE 
                    {{
                         ?entity wdt:P31|wdt:P31/wdt:P279 wd:{};
                                 wdt:P31 ?class.
                    }}
                    GROUP BY ?entity
                    OFFSET {}
                    LIMIT {}
                    """.format(wikidata_code, offset*limit, limit)
            if offset == 0:
                print(query)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            offset += 1
            if len(sparql.query().convert()['results']['bindings']) > 0:
                result = sparql.query().convert()['results']['bindings']
                results_classes[label].append(result)
                print(f'{len(results_classes[label][-1])} of results returned at offset {offset}')
            else:
                print(f'No more results returned (offset {offset})')
                break
        except Exception as e:
            if str(e).startswith('EndPointInternalError'):
                e = 'Wikidata TimeoutException'
                offset -= 1
                time.sleep(30)
            print(f'Exception at label {label} (offset {offset}) : {e}')

Organization Q43229
Name Q82799
Artist Q483501
Geolocation Q2221906

                    SELECT distinct  ?entity (GROUP_CONCAT(?class ; SEPARATOR = ';') as ?classes)
                    WHERE 
                    {
                         ?entity wdt:P31|wdt:P31/wdt:P279 wd:Q2221906;
                                 wdt:P31 ?class.
                    }
                    GROUP BY ?entity
                    OFFSET 0
                    LIMIT 10000
                    
10000 of results returned at offset 1
10000 of results returned at offset 2
10000 of results returned at offset 3
10000 of results returned at offset 4
10000 of results returned at offset 5
10000 of results returned at offset 6
10000 of results returned at offset 7
10000 of results returned at offset 8
10000 of results returned at offset 9
10000 of results returned at offset 10
10000 of results returned at offset 11
10000 of results returned at offset 12
10000 of results returned at offset 13
10000 of results returned

Exception at label Geolocation (offset 132) : Wikidata TimeoutException
Exception at label Geolocation (offset 133) : Remote end closed connection without response
10000 of results returned at offset 134
Exception at label Geolocation (offset 134) : Wikidata TimeoutException
10000 of results returned at offset 135
Exception at label Geolocation (offset 135) : Wikidata TimeoutException
Exception at label Geolocation (offset 135) : Wikidata TimeoutException
10000 of results returned at offset 136
10000 of results returned at offset 137
Exception at label Geolocation (offset 137) : Wikidata TimeoutException
10000 of results returned at offset 138
10000 of results returned at offset 139
Exception at label Geolocation (offset 139) : Wikidata TimeoutException
Exception at label Geolocation (offset 139) : Wikidata TimeoutException
Exception at label Geolocation (offset 139) : Wikidata TimeoutException
10000 of results returned at offset 140
Exception at label Geolocation (offset 140) : Wikida

In [43]:
pickle.dump(results_classes, open('results_classes.pickle', 'wb'))

In [44]:
len(results_classes)

12

In [45]:
results_classes.keys()

dict_keys(['Organization', 'Name', 'Artist', 'Geolocation', 'City', 'Capital', 'Town', 'Demonym', 'Product', 'Brand', 'Georegion', 'Country'])

In [63]:
results_classes['Name'][0][1]

{'entity': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q80443'},
 'classes': {'type': 'literal',
  'value': 'http://www.wikidata.org/entity/Q16521;http://www.wikidata.org/entity/Q1040689'}}

In [59]:
entity_classes_data = []

for class_label in tqdm(results_classes):
    results = [x for l in results_classes[class_label] for x in l]
    for result in results:
        entity_classes_data.append({
            'entity_qid': result['entity']['value'].split('/')[-1],
            'entity_classes': ';'.join([s.split('/')[-1] for s in result['classes']['value'].split(';')]),
        })

In [60]:
df_entity_classes = pd.DataFrame(entity_classes_data)
df_entity_classes

,entity_qid,entity_classes
0,Q65968,Q164950
1,Q68330,Q11032;Q11032
2,Q68682,Q7275;Q465613;Q3024240;Q7275;Q465613;Q3024240;...
3,Q69119,Q7278;Q7278
4,Q70738,Q7278;Q7278
...,...,...
3321818,Q224,Q6256;Q179164;Q3624078;Q51576574;Q6256;Q179164...
3321819,Q225,Q3624078
3321820,Q230,Q3624078
3321821,Q228,Q3624078


In [62]:
df_entity_classes.to_csv('csv/entity_classes.csv', index=False)